In [1]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
import pandas as pd
from urllib.parse import urlencode,urlparse,parse_qsl
import datetime as dt
import re

## for google sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from openpyxl.utils.cell import get_column_letter
from pathlib import Path
from urllib.parse import urlencode,urlparse,parse_qsl,quote_plus,quote

In [2]:
from utils import *

In [3]:

from OpenExcel import OpenExcel

In [4]:
from sheet_utils import *

In [5]:
from tqdm import tqdm

In [6]:
from openpyxl import load_workbook,worksheet
import pandas as pd
import datetime as dt
import os
import re
from itertools import chain
from openpyxl.utils.cell import get_column_letter

In [7]:
from sheet_utils import authorize_gspread,get_sheets
from settings import *

In [8]:
## links

# BASE_DIR= Path(__name__).resolve().parent.parent
BASE_DIR= Path.home()/ "Desktop" / "HRIS" #desktop


## for internal report Path
#HR_PHOTO_PATH=BASE_DIR / "bank.xlsx"


if not BASE_DIR.exists():
    BASE_DIR.mkdir(parents=True,exist_ok=True)

#create links

# if not REPORT_DIR.exists():
#     REPORT_DIR.mkdir(parents=True,exist_ok=True)
# if not BASE_DIR.exists():
#     BASE_DIR.mkdir(parents=True,exist_ok=True)
# if not CITIZEN_PATH.exists():
#     CITIZEN_PATH.mkdir(parents=True,exist_ok=True)
# if not KHALTI_DOC_PATH.exists():
#     KHALTI_DOC_PATH.mkdir(parents=True,exist_ok=True)   
# if not KHALTI_ID_PATH.exists():
#     KHALTI_ID_PATH.mkdir(parents=True,exist_ok=True)
# if not KHALTI_PROFILE_PATH.exists():
#     KHALTI_PROFILE_PATH.mkdir(parents=True,exist_ok=True)
# if not KHALTI_SHOPS_PATH.exists():
#     KHALTI_SHOPS_PATH.mkdir(parents=True,exist_ok=True)
# if not KHALTI_SHOPS_PATH.exists():
#     KHALTI_PAN_PATH.mkdir(parents=True,exist_ok=True)
# if not QR_DOC_PATH.exists():
#     QR_DOC_PATH.mkdir(parents=True,exist_ok=True)

In [38]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from openpyxl.utils.cell import get_column_letter

def get_worksheet_details(worksheet):
    existing_data=worksheet.get_all_values()
    LAST_LINE=len(existing_data)
    return LAST_LINE
    
def authorize_gspread(TOKEN_PATH):
    ## must store credentials 
    
    SCOPES = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',
     "https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
    if TOKEN_PATH.exists():
        creds=ServiceAccountCredentials.from_json_keyfile_name(TOKEN_PATH, SCOPES)
        return gspread.authorize(creds)
    else:
        print("could not authorize")


def update_sheets(
    sheet_filename,
    work_sheet_name,
    TITLE_ROW,
    updated_data,
    TOKEN_PATH
                 ):
    client=authorize_gspread(TOKEN_PATH)
    spreadsheet=client.open(sheet_filename)
    worksheet=spreadsheet.worksheet(work_sheet_name)
    space=1
    
    if isinstance(updated_data,pd.DataFrame):
        print("true dat")
        columns=updated_data.columns
        updated_data=updated_data.astype(str)
        data=updated_data.values.tolist()

    columns_number=len(data[0])
    LAST_LINE=get_worksheet_details(worksheet)  
    range_name=f'A{LAST_LINE+1+space}:{get_column_letter(columns_number)}{(LAST_LINE+1+space)+len(data)}'
    print(range_name)
    worksheet.update(range_name,data)
    
def get_sheets(
    sheet_filename,
    work_sheet_name,
    TITLE_ROW,
    TOKEN_PATH
):
    
    client=authorize_gspread(TOKEN_PATH)
    spreadsheet=client.open(sheet_filename)
    worksheet=spreadsheet.worksheet(work_sheet_name)
    all_data=worksheet.get_all_values()
    titles=all_data[TITLE_ROW-1]
    data=all_data[TITLE_ROW:]
    return pd.DataFrame(data,columns=titles)

In [10]:
df_storage=get_sheets(
    'Storage_',
    'Sheet1',
    1,
    TOKEN_PATH
)

In [12]:
filt=df_storage['data-Name']==''
df_storage=df_storage.loc[~filt]
df_storage

,data-start,data-today,data-deviceid,data-Name,data-Year_of_Birth,data-Gender,data-address_1_-_city,data-address_1_-_street,data-Permanent_Address_District,data-phone_1_-_Value,...,data-Helper_Level,data-Coordinator_Level,data-Flyer_Distributor_level,data-Event_Volunteer_level,data-Front_Line_Survey_level,data-Model_Height,data-Notes,data-Screened_By,data-reference_no,data-meta-instanceID
0,01/01/19,01/01/19,Manual,Aaliya Saha,null,Female,Biratnagar,Biratnagar,Biratnagar,9814300021,...,,,,,,,,Unspecified,,
1,01/01/19,01/01/19,Manual,Aarif Poudel,null,male,Unspecified,Unspecified,Unspecified,null,...,,,,,,,,Unspecified,,
2,01/01/19,01/01/19,Manual,Aaroj Sunuwar,null,male,Unspecified,Unspecified,Unspecified,null,...,,,,,,,,Unspecified,,
3,01/01/19,01/01/19,Manual,Aashika Bashyal,null,Female,Butwal,Butwal,Butwal,9846014632,...,,Level_3,,,,,,Unspecified,,
4,01/01/19,01/01/19,Manual,Aashish Rai,null,male,Lalitpur,Imadol,Unspecified,9808105714,...,level_2,,level_2,level_2,level_2,,#Lazy,Unspecified,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,19/03/22 10:48,19/03/22,collect:J3BDiwjAF6qjR9fb,Bipin joshi,01/01/99,male,Ktm,Lokanthali,Darchula,9865656548,...,,,,,,,#brandpromoter,Apsara,,uuid:961bfbae-3c69-43b5-baad-d0041b6e482d
1881,19/03/22 11:48,19/03/22,collect:J3BDiwjAF6qjR9fb,Rijan maharjan,01/01/02,male,Ktm,Bafal,Bafal,9823530599,...,,,,,,,#raider,Apsara,,uuid:b9d6a5f7-e068-40b9-afb1-36f61ec1d789
1882,19/03/22 11:51,19/03/22,collect:J3BDiwjAF6qjR9fb,Rasun maharjan,01/01/01,male,Ktm,Kalanki,Kalanki,9864676200,...,,,,,,,#raider,Apsara,,uuid:afbb150a-0207-4112-b0e0-1f3edd0cdbf5
1883,19/03/22 12:28,19/03/22,collect:J3BDiwjAF6qjR9fb,Nabina regmi,01/01/92,male,Ktm,Sanepa,Dhading,9861888765,...,,,,,,,#raider+brandpromoter,Apsara,,uuid:6481f75b-959c-4ba7-b510-1ab7ff34afd2


In [16]:
df_storage.columns

Index(['data-start', 'data-today', 'data-deviceid', 'data-Name',
       'data-Year_of_Birth', 'data-Gender', 'data-address_1_-_city',
       'data-address_1_-_street', 'data-Permanent_Address_District',
       'data-phone_1_-_Value', 'data-phone_2_-_Value',
       'data-Secondary_Contact_Person', 'data-phone_3_-_Value', 'data-e-mail',
       'data-Short_Introduction', 'data-audio_introduction', 'data-photo',
       'data-Driving_License', 'data-Other_License_Categories',
       'data-Possession_of_Any_Vehicle', 'data-Vehicle_Type',
       'data-Transfer_To', 'data-Reason_for_N_A', 'data-Emcee_Level',
       'data-Brand_Promoter_Level', 'data-Model_Promoter_Level',
       'data-Helper_Level', 'data-Coordinator_Level',
       'data-Flyer_Distributor_level', 'data-Event_Volunteer_level',
       'data-Front_Line_Survey_level', 'data-Model_Height', 'data-Notes',
       'data-Screened_By', 'data-reference_no', 'data-meta-instanceID'],
      dtype='object')

In [20]:
ENROLL_FILE_PATH=Path('C:\Dropbox\Dropbox\PTK Nepal\Operation\Finance\TeSalary\Te salary.xlsx')

In [18]:
df_storage['data-Driving_License'].unique()

array(['Unspecified', 'no', 'category_a__motorcycle_scooter',
       'category_a__motorcycle_scooter, category_b__car_jeep_delivery',
       'category_k__scooter__moped', '', 'yes',
       'no, category_a__motorcycle_scooter',
       'category_a__motorcycle_scooter, no',
       'category_b__car_jeep_delivery',
       'category_b__car_jeep_delivery, category_a__motorcycle_scooter'],
      dtype=object)

array(['idk', 'False', 'A', 'AB', 'K', 'True', 'B'], dtype=object)

In [21]:
df_enroll=OpenExcel(filename=ENROLL_FILE_PATH).from_table('data')

In [22]:
df_enroll.columns

Index(['Name', 'Sum of amount', 'Sum of Payment', 'Project', 'Column1',
       'Column2', 'Column3'],
      dtype='object')

In [23]:
df_enroll=df_enroll.groupby(['Name']).agg({'Sum of amount':'sum'}).reset_index()

In [24]:
d=pd.merge(df_storage,df_enroll,how='left',left_on='data-Name',right_on='Name') ## df finalized
d.columns

Index(['data-start', 'data-today', 'data-deviceid', 'data-Name',
       'data-Year_of_Birth', 'data-Gender', 'data-address_1_-_city',
       'data-address_1_-_street', 'data-Permanent_Address_District',
       'data-phone_1_-_Value', 'data-phone_2_-_Value',
       'data-Secondary_Contact_Person', 'data-phone_3_-_Value', 'data-e-mail',
       'data-Short_Introduction', 'data-audio_introduction', 'data-photo',
       'data-Driving_License', 'data-Other_License_Categories',
       'data-Possession_of_Any_Vehicle', 'data-Vehicle_Type',
       'data-Transfer_To', 'data-Reason_for_N_A', 'data-Emcee_Level',
       'data-Brand_Promoter_Level', 'data-Model_Promoter_Level',
       'data-Helper_Level', 'data-Coordinator_Level',
       'data-Flyer_Distributor_level', 'data-Event_Volunteer_level',
       'data-Front_Line_Survey_level', 'data-Model_Height', 'data-Notes',
       'data-Screened_By', 'data-reference_no', 'data-meta-instanceID', 'Name',
       'Sum of amount'],
      dtype='object')

In [13]:
def get_fn(x):
    fn=x.split(' ')[0]
    return fn
def get_ln(x):
    ln=x.split(' ')[-1]
    return ln

def get_pic_id(x):
    try:
        id_name=x.split("=")[1]
        return id_name
    except (IndexError, AttributeError):
        return pd.NA
    

In [51]:
df=pd.DataFrame()
df=df.assign(
    id=df_storage['data-meta-instanceID'],
    email=df_storage['data-e-mail'].replace({'Unspecified':pd.NA}),
    phone=df_storage['data-phone_1_-_Value'].apply(lambda x:'977'+ x if x!='null' else pd.NA),
    name=df_storage['data-Name'],
    ct=df_storage['data-address_1_-_city'].apply(str.lower).replace({'unspecified':pd.NA}).values,
    country='NP',
    dob=df_storage['data-Year_of_Birth'],
    today=df_storage['data-today'],
    photo_id=[get_pic_id(photo) for photo in df_storage['data-photo'].values],
    fn=lambda d: d['name'].apply(get_fn),
    ln=lambda d: d['name'].apply(get_ln),
    lisc=df_storage['data-Driving_License'].replace({
    'Unspecified':'idk', 'no':'False', 'category_a__motorcycle_scooter':'A',
       'category_a__motorcycle_scooter':'A', 'category_b__car_jeep_delivery':"B",
       'category_k__scooter__moped':"K", '':'idk', 'yes':'True',
       'category_a__motorcycle_scooter':"A",
       'category_a__motorcycle_scooter, no':"A",
       'category_b__car_jeep_delivery':"B",
       'category_b__car_jeep_delivery':"B", "category_a__motorcycle_scooter":"A",
       'no, category_a__motorcycle_scooter':"A",
        'category_b__car_jeep_delivery, category_a__motorcycle_scooter':'AB',
        'category_a__motorcycle_scooter, category_b__car_jeep_delivery':'AB',
})
)

In [58]:
existing_pics=pd.read_csv("done.csv").id.values
[True if pic in existing_pics and pic=='nan' else False for pic in df['id']]

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [18]:
#filter useless
empty_id_filter=df['photo_id'].isna()
df_filtered=df.loc[~empty_id_filter]
file_ids=df_filtered['photo_id'].values

def assign_filename(df_filtered):
    lst=[]
    count=0
    for name,phone in zip(
                    df_filtered['name'],
                    df_filtered['phone']):
        count+=1        
        lst.append(f'{count}_{name}_{phone}.jpg')
    return np.array(lst)

file_names=assign_filename(df_filtered)

array(['1_Sirjana bishwokarma_9779847604758.jpg',
       '2_Yogesh Shrestha_9779823693115.jpg',
       '3_Subrina Chitrakar_9779864836781.jpg',
       '4_Anup Thapa_9779861901337.jpg',
       '5_Nikita shrestha_9779861927443.jpg',
       '6_Bijay Acharya_9779812405698.jpg',
       '7_Surya Bhattarai_9779811606901.jpg',
       '8_Anil Budathoki_9779860538575.jpg',
       '9_Pradish shrestha_9779803207887.jpg',
       '10_Nisha thakuri_9779863913478.jpg',
       '11_Sapath raj shrestha_9779823291342.jpg',
       '12_Binesh Ghalan_9779818074122.jpg',
       '13_Raj Kumar sharma_9779880851932.jpg',
       '14_Nabin khatiwada_9779849732310.jpg',
       '15_Shubham khanal_9779867208395.jpg',
       '16_Sanjay lal waiba_9779803046965.jpg',
       '17_Samit Maharjan_9779843405365.jpg',
       '18_Aakash shrestha_9779813204575.jpg',
       '19_Shreya Shahi_9779818428328.jpg',
       '20_Bijay Karki_9779808650010.jpg',
       '21_Anjila Maharjan_9779841249258.jpg',
       '22_Arpan Acharya_97798

In [78]:
df_filtered

,email,phone,name,ct,country,dob,today,photo_id,fn,ln
801,bishwokarmasirjana12@gmail.com,9779847604758,Sirjana bishwokarma,kathmandu,NP,01/01/97,18/10/19,1Dgl1gy1q_aojnJr7RcSd6ssE5bmIaTzB,Sirjana,bishwokarma
802,Bijayasharma34@gmail.com,9779823693115,Yogesh Shrestha,kathmandu,NP,01/01/00,23/10/19,1X3fSXiej4_AAcdbD1xvD7SWjrEEj0tom,Yogesh,Shrestha
803,Subrina.chitrakar81@gmail.com,9779864836781,Subrina Chitrakar,lalitpur,NP,01/01/99,23/10/19,1HX04IN9UOoBZjt7myWFoEYaBaSC81dyp,Subrina,Chitrakar
804,Anup.thapa0318@gmail.com,9779861901337,Anup Thapa,ktm,NP,01/01/99,22/10/19,1iAiVCsw60iCNdu2UoI6FdaYaQtVjnbR2,Anup,Thapa
805,nikitaghatsn@gmail.com,9779861927443,Nikita shrestha,kathmandu,NP,01/01/19,22/10/19,1-xY-usChaF-To5yncz_r1jvyI3-cAodc,Nikita,shrestha
...,...,...,...,...,...,...,...,...,...,...
1731,Sanamadhikari765@gmail.com,9779745711933,Sanam adhikari,ktm,NP,01/01/03,03/03/22,1bCLdeEL_7qrct03nuxfSmdXEF30wO6yZ,Sanam,adhikari
1732,prayaskharel175@gmail.com,9779824931143,Prayas Kharel,kathmandu,NP,01/01/03,03/03/22,1PjKTMWZ-eXXOSf_afwMINHRVtfZb85Mz,Prayas,Kharel
1733,Sushilstha101@gmail.com,9779849840273,Sushil shrestha,ktm,NP,01/01/93,03/03/22,1LNRuDqMUKx58MOwE883JtZEM2zjfJgbN,Sushil,shrestha
1734,Nitishsahane10@gmail.com,9779807111077,Nitish shahani,ktm,NP,01/01/02,03/03/22,17G5RF9PcCpLajQEe6WX_iHxPn-ivrox6,Nitish,shahani


In [ ]:
def get_link(x):
    front_string="https://drive.google.com/file/d/"
    id_name=x.split("=")[1]
    endpoints=f'https://drive.google.com/u/0/uc'
    params=urlencode({'id':id_name})
    url=f'{endpoints}?{params}&export=download'
    return url

In [33]:
df['ct']=d['data-address_1_-_city'].apply(str.lower)

In [34]:
df['ct']=df['ct'].replace({'unspecified':pd.NA}).values

In [35]:
df['ct'].unique()

array(['biratnagar', <NA>, 'butwal', 'lalitpur', 'kathmandu', 'birtamode',
       'wrong number', 'bhaktapur', 'pokhara', 'jhapa', 'bharatpur',
       'dhngadi', 'tanahun', 'surkhet', 'ilam', 'nepalgunj', 'birgunj',
       'dharan', 'udypur', 'ktm', 'kathamandu', 'udayapur', 'paiyatat',
       'kirtipur', 'baneshwor', '', 'itahari', 'hetauda', 'chhabahil',
       'chitwan', 'sindupalchok', 'manamaiju kathmandu', 'kanchanpur',
       'kailali', 'bkt', 'banepa', 'ktm budhanilkantha', 'jadibuti',
       'bktm', 'lokanthali', 'narayanghat', '#ktm'], dtype=object)

In [18]:
df['data-photo'].values

array(['Unspecified', 'Unspecified', 'Unspecified', ...,
       'https://drive.google.com/open?id=1LNRuDqMUKx58MOwE883JtZEM2zjfJgbN',
       'https://drive.google.com/open?id=17G5RF9PcCpLajQEe6WX_iHxPn-ivrox6',
       'https://drive.google.com/open?id=1iJlIyAdFfOwSgXA2koittsm8t_JPuRSH'],
      dtype=object)

In [40]:
def get_pic_id(x):
    try:
        id_name=x.split("=")[1]
        return id_name
    except (IndexError, AttributeError):
        return pd.NA
    

In [48]:
[get_pic_id(photo) if (photo!='Unspecified') or (photo!= pd.NA) else pd.NA for photo in df['data-photo'].values]

TypeError: boolean value of NA is ambiguous

In [36]:
df['data-photo'].values

array(['Unspecified', 'Unspecified', 'Unspecified', ...,
       'https://drive.google.com/open?id=1LNRuDqMUKx58MOwE883JtZEM2zjfJgbN',
       'https://drive.google.com/open?id=17G5RF9PcCpLajQEe6WX_iHxPn-ivrox6',
       'https://drive.google.com/open?id=1iJlIyAdFfOwSgXA2koittsm8t_JPuRSH'],
      dtype=object)

In [52]:
[get_pic_id(photo) for photo in df['data-photo'].values]

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [39]:
df['dob'].values

array(['null', 'null', 'null', ..., '01/01/99', '01/01/03', '01/01/02'],
      dtype=object)

In [40]:
df['today']=d['data-today']

In [43]:
def get_fn(x):
    fn=x.split(' ')[0]
    return fn

#df['fn']=df['name'].apply(get_fn)

In [45]:
def get_ln(x):
    ln=x.split(' ')[-1]
    return ln

In [43]:
df['ln']=df['name'].apply(get_ln)

In [44]:
df['amount']=d['Sum of amount']

In [45]:
import matplotlib.pyplot as plt


In [46]:
# sns.kdeplot(df['amount'])

In [47]:
import scipy.stats as stats
df['amount']=df['amount'].replace({np.NaN:0.0})
df['amount_std']=stats.zscore(df['amount'])

In [48]:
# retention
import datetime as dt

In [49]:
today=dt.datetime.today().date()

In [50]:
today

datetime.date(2022, 3, 3)

In [51]:
df['retention']=(dt.datetime.today()-pd.to_datetime(df['today'])).dt.days
df['retention_std']=stats.zscore(df['retention'])

In [52]:
def get_value(amt,ret):
    mu=-ret + amt
    return mu
df['value']=get_value(df['amount_std'].values,df['retention_std'].values)

In [53]:
df['value']=stats.zscore(df['value'])

In [54]:
## lets filter

patt=re.compile(r'kathmandu|bhaktapur|ktm|kathamandu|bkt|patan|lalitpur')
lst=[]
def match_valley(x):
    if re.match(patt,x):
        return True
    else:
        return False

In [55]:
filt=[match_valley(i) if i is not pd.NA else True for i in df['ct']]
df=df.loc[filt]

In [56]:
df['vals']=df['value']-df['value'].min()+0.1
df.to_csv('audience.csv',index=None)

In [57]:
df['value'].min()

-0.7940223738716448

In [58]:
d.to_csv('storage.csv')

In [59]:

df_storage['data-photo'].values

array(['Unspecified', 'Unspecified', 'Unspecified', ...,
       'https://drive.google.com/open?id=1k61V-R-QCq0GW4zq4oIOKSEF3oSrg8NH',
       'https://drive.google.com/open?id=1-CEiRsYFMavTG_UXx_AOLIFhK8gMg_8M',
       'https://drive.google.com/open?id=189JyJ5hilS-6-binp4K8pmluRak4xCnb'],
      dtype=object)

In [60]:
def get_link(x):
    front_string="https://drive.google.com/file/d/"
    id_name=x.split("=")[1]
    endpoints=f'https://drive.google.com/u/0/uc'
    params=urlencode({'id':id_name})
    url=f'{endpoints}?{params}&export=download'
    return url

def get_pic():
    df=df_storage.loc[df_storage['data-photo'] != "Unspecified",]
    count=0
    for name, link,number in zip(df['data-Name'],df['data-photo'],df['data-phone_1_-_Value']):
#         if count==5:
#             break
        count+=1
        formatted_link=get_link(link)
        print(url)
        r=requests.get(formatted_link,headers=None,stream=True)

        image=Image.open(io.BytesIO(r.content))
        #image.save(BASE_DIR / f'{count}_{name}_{number}.jpg')

def get_filename(x):
    id_name=x.split("=")[1]

    return id_name

In [61]:
link="https://drive.google.com/file/d/1Mi4UVRZfaf5KDDgFis1Sj1m-vOajIPta/viewhttps://drive.google.com/file/d/1DCT0gDvuHPKbeCKp_jhIR98NgEyFKE7J/view?usp=sharing"

In [66]:
link='https://drive.google.com/open?id=1HWxARUS1jaVe4c9MOUvgC_kfEasndjTR'
get_filename(link)

'1HWxARUS1jaVe4c9MOUvgC_kfEasndjTR'

In [63]:
import requests

In [64]:
get_pic()

NameError: name 'url' is not defined

In [65]:
from PIL import Image
import io

In [67]:
import time